In [2]:
import pandas as pd
import numpy as np
import os
import sys
from rdkit import Chem
from rdkit.Chem import AllChem

# 读取PDB文件
def read_pdb(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines

# 遍历本文件夹下的PDB文件
def get_pdb_files():
    files = []
    for file in os.listdir():
        if file.endswith('.pdb'):
            files.append(file)
    # 按照文件名中的数字排序
    files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))
    return files

# 读取PDB文件中原子信息和坐标信息
def get_atom_coord(lines):
    atom_coord = []
    oxygen_coord = []
    for line in lines:
        if line.startswith('ATOM') or line.startswith('HETATM'):
            atom = line[12:16].strip()
            atom = ''.join(filter(str.isalpha, atom))
            try:
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
            except ValueError as e:
                print(f"Error parsing coordinates in line: {line}")
                print(e)
                continue
            atom_coord.append([atom, x, y, z])
            if atom.startswith('O'):
                oxygen_coord.append([atom, x, y, z])
    return atom_coord, oxygen_coord

# 将PDB文件转换为SMILES并判断是否存在C=O
def pdb_to_smiles_and_check_carbonyl(file):
    lines = read_pdb(file)
    atom_coord, oxygen_coord = get_atom_coord(lines)
    carbonyl_oxygen_coord = []
    # 将PDB文件转换为RDKit分子对象
    mol = Chem.MolFromPDBFile(file, removeHs=False)
    if mol is None:
        print(f"Failed to read molecule from {file}")
        return None, None
    
    # 转换为SMILES
    smiles = Chem.MolToSmiles(mol)
    
    # 查找C=O键
    patt = Chem.MolFromSmarts('C=O')
    matches = mol.GetSubstructMatches(patt)
    
    for match in matches:
        carbon_idx, oxygen_idx = match
        pos = mol.GetConformer().GetAtomPosition(oxygen_idx)
        carbonyl_oxygen_coord.append(['O', pos.x, pos.y, pos.z])
    
    return smiles, carbonyl_oxygen_coord

# 在羰基氧附近添加一个Li原子, 距离为3埃
def add_lithium_near_carbonyl(file, carbonyl_oxygen_coord):
    if not carbonyl_oxygen_coord:
        print(f"No carbonyl oxygen found in {file}")
        return
    
    # 读取PDB文件内容
    lines = read_pdb(file)
    
    # 找到最后一个HETATM行的索引
    last_hetatm_index = max(i for i, line in enumerate(lines) if line.startswith('HETATM'))
    
    # 获取羰基氧的坐标
    ox, oy, oz = carbonyl_oxygen_coord[0][1:]
    
    # 确保Li原子远离所有C H的同时, 该Li原子与羰基氧的距离为3埃
    def is_far_from_ch(x, y, z, threshold=2.0):
        for line in lines:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                atom = line[12:16].strip()
                atom = ''.join(filter(str.isalpha, atom))
                if atom in ['C', 'H']:
                    cx = float(line[30:38].strip())
                    cy = float(line[38:46].strip())
                    cz = float(line[46:54].strip())
                    distance = np.sqrt((x - cx)**2 + (y - cy)**2 + (z - cz)**2)
                    if distance < threshold:
                        return False
        return True
    
    # 尝试在不同方向上添加Li原子，直到找到一个远离所有C H的坐标
    directions = [(0, 0, 3), (0, 0, -3), (3, 0, 0), (-3, 0, 0), (0, 3, 0), (0, -3, 0)]
    for dx, dy, dz in directions:
        li_x, li_y, li_z = ox + dx, oy + dy, oz + dz
        if is_far_from_ch(li_x, li_y, li_z):
            break
    else:
        print(f"Could not place Li atom far from C and H in {file}")
        return
    
    # 创建新的HETATM行
    li_atom_line = f"HETATM{len(lines):>5}  LI   UNL     1    {li_x:>8.3f}{li_y:>8.3f}{li_z:>8.3f}  1.00  0.00          LI  \n"
    
    # 插入新的HETATM行
    lines.insert(last_hetatm_index + 1, li_atom_line)
    
    # 写入新的PDB文件
    output_file = file.replace('.pdb', '_with_Li.pdb')
    with open(output_file, 'w') as f:
        f.writelines(lines)
    
    print(f"Added Li atom near carbonyl oxygen in {output_file}")


files = get_pdb_files()

for file in files: 
    # 调用函数在羰基氧附近添加Li原子
    smiles, carbonyl_oxygen_coord = pdb_to_smiles_and_check_carbonyl(file)
    add_lithium_near_carbonyl(file, carbonyl_oxygen_coord)

No carbonyl oxygen found in ext_0.pdb
No carbonyl oxygen found in ext_1.pdb
No carbonyl oxygen found in ext_2.pdb
No carbonyl oxygen found in ext_3.pdb
